In [39]:
import subprocess
import time
import datetime
import collections
import numpy as np
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

In [47]:
proc = subprocess.Popen(['pwd'],stdout=subprocess.PIPE)

out = proc.communicate()

print(out)

(b'/home/jovyan/work\n', None)


In [5]:
def sample_func(process_index):
    start = time.time()
    print('process index: %s started.' % process_index)
    num = 0

    for i in range(10000000):
        num += 1
    end = time.time()
    print('process index: %s ended.' % process_index)
    print('%s' % (end - start))

In [18]:
from multiprocessing import Manager, Process

def sample_func(initial_num, returned_dict, process_index):
    for i in range(100):
        initial_num += 1
    returned_dict[process_index] = initial_num


manager = Manager()
returned_dict = manager.dict()
process_list = []

for i in range(100):
    process = Process(
        target=sample_func,
        kwargs={
            'initial_num': 50,
            'returned_dict': returned_dict,
            'process_index': i,
        })

    process.start()
process.join()

In [154]:
# 長めな処理
def gcd(pair):
    a, b = pair
    if a < b:
        low = a
    else:
        low = b
    # print("a: %f b: %f" %(a, b))
    for i in range(int(low), 0, -1):
        if a % i == 0 and b % i ==0:
            return i

# 逐次処理
def single_thread_test(sample_func, arr_list):
    result_1 = []
    for i in range(10):
        start = time.time()
        results = list(map(sumple_func, arr_list))
        end = time.time()
        # print('Took %.3f seconds' %( end - start))
        result_1.append(end - start)
    return result_1

    
# ThreadPoolExecutor
def thread_pool_test(sample_func, arr_list):
    result_2 = []
    for i in range(10):
        start = time.time()
        pool = ThreadPoolExecutor(max_workers=2)
        results = list(pool.map(sample_func, arr_list))
        end = time.time()
        # print('Took %.3f seconds' %( end - start))
        result_2.append(end - start)
    return result_2


# ProcessPoolExecutor
def process_pool_test(sample_func, arr_list):
    result_3 = []
    for i in range(10):
        start = time.time()
        pool = ProcessPoolExecutor(max_workers=10)
        results = list(pool.map(sample_func, arr_list))
        end = time.time()
        # print('Took %.3f seconds' %( end - start))
        result_3.append(end - start)
    return result_3

In [139]:
# 処理の個数
num_process_list = [5, 10, 100, 1000, 10000]
# 処理の重さ
weight_process_list = [100, 10000, 1000000]
# 検証用のデータ
arr_obj = {}
for index in range(10):
    arr_list = []
    for value in range(10):
        arr_list.append(np.random.rand(2) * 10000)
    arr_obj[index] = arr_list